In [7]:
import regex 
import pandas as pd
import bs4
import requests
import spacy 
import csv
from spacy import displacy 
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher
from spacy.tokens import Span 

import urllib.request
from bs4 import BeautifulSoup

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)

In [4]:
# indicate address of chosen url - text data to be scraped
url = "https://www.sciencefocus.com/the-human-body/iq-score-measures"

# open the url for reading
html = urllib.request.urlopen(url)

# Parse the open html file
htmlParse = BeautifulSoup(html, 'html.parser')

parsed_text = ""

for para in htmlParse.find_all('p'): 
    parsed_text = " ".join((parsed_text, str(para.get_text())))

sentences = [[i] for i in nlp(parsed_text).sents]


In [5]:
parsed_text

" IQ is useful for measuring a specific cognitive skillset but there's other ways to be smart. Photo credit: Getty Dr Peter Bentley In 1904, English psychologist Charles Spearman discovered there was a strange link between certain mental skills, such as mathematics, verbal fluency, spatial visualisation and memory.\xa0 He found that people who did well at one tended to do well at others, while those who did poorly at one would do poorly at others. And his finding has been shown to be true so many times that it has been called “arguably the most replicated result in all psychology”.\xa0 By using statistics, it’s possible to derive a single factor, known as general intelligence (g), that indicates the level of general cognitive ability compared to other people. This general intelligence has been divided into two types.\xa0 The first is fluid intelligence (gf), which doesn’t depend on formal education and is more about solving abstract reasoning problems. The second, crystallised intellig

In [6]:
sentences

[[ IQ is useful for measuring a specific cognitive skillset but there's other ways to be smart.],
 [Photo credit: Getty Dr Peter Bentley In 1904, English psychologist Charles Spearman discovered there was a strange link between certain mental skills, such as mathematics, verbal fluency, spatial visualisation and memory.  ],
 [He found that people who did well at one tended to do well at others, while those who did poorly at one would do poorly at others.],
 [And his finding has been shown to be true so many times that it has been called “arguably the most replicated result in all psychology”.  ],
 [By using statistics, it’s possible to derive a single factor, known as general intelligence (g), that indicates the level of general cognitive ability compared to other people.],
 [This general intelligence has been divided into two types.  ],
 [The first is fluid intelligence (gf), which doesn’t depend on formal education and is more about solving abstract reasoning problems.],
 [The second

In [8]:
myheaders = ['sentence']
myvalues = sentences 
filename = 'article_text.csv'
with open(filename, 'w', newline='') as myfile: 
    writer = csv.writer(myfile)
    writer.writerow(myheaders)
    writer.writerows(myvalues)

In [12]:
df = pd.read_csv('article_text.csv')
df

,sentence
0,IQ is useful for measuring a specific cognitive skillset but there's other ways to be smart.
1,"Photo credit: Getty Dr Peter Bentley In 1904, English psychologist Charles Spearman discovered there was a strange link between certain mental skills, such as mathematics, verbal fluency, spatial ..."
2,"He found that people who did well at one tended to do well at others, while those who did poorly at one would do poorly at others."
3,And his finding has been shown to be true so many times that it has been called “arguably the most replicated result in all psychology”.
4,"By using statistics, it’s possible to derive a single factor, known as general intelligence (g), that indicates the level of general cognitive ability compared to other people."
5,This general intelligence has been divided into two types.
6,"The first is fluid intelligence (gf), which doesn’t depend on formal education and is more about solving abstract reasoning problems."
7,"The second, crystallised intelligence (gc), is more about learned experience, general information and vocabulary."
8,"Researchers claim that gf peaks at around age 20 and then declines, while gc stays stable or improves with age."
9,It’s known that g is heritable – you’re likely to inherit mental skills from your parents.


In [13]:
def get_entities(sent): 
    # chunk 1: defining the variables used within the function (local variables), which store filtered words that will be classed as the entity pair. 
    ent1 = ""
    ent2 = ""
    # these variables are empty to begin with, but are updated at the end of each for loop  
    prv_tok_dep = ""
    prv_tok_text = ""
    # two grammatical elements which are incorporated into the final entity, if present. 
    prefix = ""
    modifier = ""

# the NLP model provides metadata on the text data e.g. dep_, .lemma, 
    for tok in nlp(sent): 
        # chunk 2: a for loop runs through each word in the sentence "a.k.a token", and assigns the token to either prefix, modifier 
        if tok.dep_ != "punct": 

            if tok.dep_ == "compound": 
                prefix = tok.text
                # the prefix is updated to the token (tok.text) if the token.dep_ is a compound 
                if prv_tok_dep == "compound": 
                # if the previous token (off-by-one) is a compound, the prefix takes on two tokens.  
                    prefix = prv_tok_dep + " " + tok.text
            # if the token dependency ends with mod then the modifier is assigned the tok.text 
            if tok.dep_.endswith("mod") == True:
                modifier = tok.text

                if prv_tok_dep == "compound": 
                    modifier = prv_tok_dep + " " + tok.text 
            # the first entity is the subject. This if loop identifies the subject token through dependency parsing functionality of spaCy.
            if tok.dep_.find("subj") == True: 
                ent1 = modifier + " " + prefix + " " + tok.text  
                # Empty variables when the subject token is found. 
                prefix = ""
                modifier = ""
                prv_tok_dep = ""
                prv_tok_text = ""
            # If the token dependency is object, assign token text to entity two 
            if tok.dep_.find("obj") == True: 
                ent2 =  modifier + " " + prefix + " " + tok.text   

            # The variables are updated by permanent attributes of text data that is processed using the NLP model. 
            prv_tok_dep = tok.dep_
            prv_tok_text = tok.text
    # Following the classification of text into one or none of the entities, return the entity pair
    return [ent1.strip(), ent2.strip()]


In [24]:
sentence = "The bananas had green and white spots"
doc = nlp(sentence)
for entity in doc: 
    print(entity.dep_) 
    


det
nsubj
ROOT
amod
cc
conj
dobj
